<a href="https://colab.research.google.com/github/tkg31302/SoC_2021/blob/main/Kaggle_Digit_Recognizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Digit Recognizer**

In [23]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"tarunkumargupta","key":"8645349be186021db50663d0118322be"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c digit-recognizer

  0% 0.00/9.16M [00:00<?, ?B/s]
100% 9.16M/9.16M [00:00<00:00, 84.2MB/s]
  0% 0.00/6.09M [00:00<?, ?B/s]
100% 6.09M/6.09M [00:00<00:00, 99.9MB/s]
  0% 0.00/235k [00:00<?, ?B/s]
100% 235k/235k [00:00<00:00, 67.3MB/s]


In [ ]:
from zipfile import ZipFile
file_name = "train.csv.zip"
with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

Done


In [ ]:
from zipfile import ZipFile
file_name = "test.csv.zip"
with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

Done


In [24]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [25]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt

In [26]:
df_train = pd.read_csv("train.csv")
x_train = df_train.drop("label", axis = 1)
y_train = df_train["label"]

x_test = pd.read_csv("test.csv")

# Preprocess the data (these are NumPy arrays)
x_train = x_train.astype("float32") / 255
x_train = x_train.values.reshape(-1, 28, 28, 1)
x_test = x_test.astype("float32") / 255
x_test = x_test.values.reshape(-1, 28, 28, 1)

y_train = y_train.astype("float32")


In [27]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
 
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
 
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])    
                           
  # Construct the layers into a model that TensorFlow can execute.  
  # Notice that the loss function for multi-class classification
  # is different than the loss function for binary classification.  
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
   
 
 
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 3, 128)         73856     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 1, 1, 128)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 1, 128)        

In [28]:
print("Fit model on training data")
history = model.fit(
    x_train,
    y_train,
    batch_size=300,
    epochs=30,
    validation_split=0.2
)

Fit model on training data
Epoch 1/30
112/112 [==============================] - 22s 196ms/step - loss: 1.5393 - accuracy: 0.4797 - val_loss: 0.2430 - val_accuracy: 0.9305
Epoch 2/30
112/112 [==============================] - 22s 195ms/step - loss: 0.3467 - accuracy: 0.8940 - val_loss: 0.1481 - val_accuracy: 0.9570
Epoch 3/30
112/112 [==============================] - 22s 195ms/step - loss: 0.2390 - accuracy: 0.9312 - val_loss: 0.1156 - val_accuracy: 0.9671
Epoch 4/30
112/112 [==============================] - 22s 194ms/step - loss: 0.1830 - accuracy: 0.9431 - val_loss: 0.0908 - val_accuracy: 0.9715
Epoch 5/30
112/112 [==============================] - 22s 193ms/step - loss: 0.1479 - accuracy: 0.9549 - val_loss: 0.0794 - val_accuracy: 0.9746
Epoch 6/30
112/112 [==============================] - 22s 193ms/step - loss: 0.1370 - accuracy: 0.9590 - val_loss: 0.0783 - val_accuracy: 0.9761
Epoch 7/30
112/112 [==============================] - 22s 193ms/step - loss: 0.1245 - accuracy: 0.9615 

In [29]:
y = model.predict(x_test)
y = np.argmax(y, axis = 1)

In [30]:
df = pd.DataFrame()
df["Label"] = y
df["ImageID"] = range(1, 28001)
df.head()

,Label,ImageID
0,2,1
1,0,2
2,9,3
3,9,4
4,3,5


In [31]:
df.to_csv("submissions.csv", index=False)

# accuracy of 98.703 on the test set given by kaggle: https://www.kaggle.com/c/digit-recognizer/
# Link for access to my kaggle notebook - https://www.kaggle.com/tarunkumargupta/notebook-tkg